In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
file_path=[]
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        file_path.append(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/vehicle-maintenance-data/vehicle_maintenance_data.csv


In [4]:
df= pd.read_csv(file_path[0])
df.head()

,Vehicle_Model,Mileage,Maintenance_History,Reported_Issues,Vehicle_Age,Fuel_Type,Transmission_Type,Engine_Size,Odometer_Reading,Last_Service_Date,Warranty_Expiry_Date,Owner_Type,Insurance_Premium,Service_History,Accident_History,Fuel_Efficiency,Tire_Condition,Brake_Condition,Battery_Status,Need_Maintenance
0,Truck,58765,Good,0,4,Electric,Automatic,2000,28524,2023-11-23,2025-06-24,Second,20782,6,3,13.622204,New,New,Weak,1
1,Van,60353,Average,1,7,Electric,Automatic,2500,133630,2023-09-21,2025-06-04,Second,23489,7,0,13.625307,New,New,Weak,1
2,Bus,68072,Poor,0,2,Electric,Automatic,1500,34022,2023-06-27,2025-04-27,First,17979,7,0,14.306302,New,Good,Weak,1
3,Bus,60849,Average,4,5,Petrol,Automatic,2500,81636,2023-08-24,2025-11-05,Second,6220,7,3,18.709467,New,Worn Out,New,1
4,Bus,45742,Poor,5,1,Petrol,Manual,2000,97162,2023-05-25,2025-09-14,Third,16446,6,2,16.977482,Good,Good,Weak,1


In [5]:
df.describe()

,Mileage,Reported_Issues,Vehicle_Age,Engine_Size,Odometer_Reading,Insurance_Premium,Service_History,Accident_History,Fuel_Efficiency,Need_Maintenance
count,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000
mean,54931.232680,2.497420,5.492260,1556.292000,75551.187060,17465.340700,5.515560,1.50156,14.990323,0.809960
std,14401.912925,1.708781,2.875682,627.677218,43088.105658,7223.393401,2.874899,1.11951,2.885583,0.392336
min,30001.000000,0.000000,1.000000,800.000000,1001.000000,5000.000000,1.000000,0.00000,10.000098,0.000000
25%,42471.500000,1.000000,3.000000,1000.000000,38009.000000,11189.750000,3.000000,0.00000,12.489037,1.000000
50%,54810.000000,2.000000,5.000000,1500.000000,75598.500000,17477.500000,6.000000,2.00000,14.986352,1.000000
75%,67391.500000,4.000000,8.000000,2000.000000,112999.500000,23692.000000,8.000000,3.00000,17.474676,1.000000
max,80000.000000,5.000000,10.000000,2500.000000,149999.000000,30000.000000,10.000000,3.00000,19.999968,1.000000


In [10]:
df.isnull().sum()

Vehicle_Model           0
Mileage                 0
Maintenance_History     0
Reported_Issues         0
Vehicle_Age             0
Fuel_Type               0
Transmission_Type       0
Engine_Size             0
Odometer_Reading        0
Last_Service_Date       0
Warranty_Expiry_Date    0
Owner_Type              0
Insurance_Premium       0
Service_History         0
Accident_History        0
Fuel_Efficiency         0
Tire_Condition          0
Brake_Condition         0
Battery_Status          0
Need_Maintenance        0
dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline



In [14]:
import pandas as pd
import numpy as np
from datetime import date
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

# ----------------------------------------------------------------------
# 1. Custom Feature Engineering Transformer (for Date Columns)
# ----------------------------------------------------------------------

class DateToDaysTransformer(BaseEstimator, TransformerMixin):
    """Calculates the number of days between a date column and a reference date."""
    def __init__(self, date_col, reference_date='2025-10-15'):
        self.date_col = date_col
        # Use a fixed reference date for consistency across train/test splits
        self.reference_date = pd.to_datetime(reference_date) 

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_copy = X.copy()
        
        # Convert to datetime objects, handling potential errors if needed
        X_copy[self.date_col] = pd.to_datetime(X_copy[self.date_col], errors='coerce')
        
        # Calculate the difference in days (will result in NaT if parsing failed)
        time_delta = self.reference_date - X_copy[self.date_col]
        
        # Create the new numerical feature and fill any NaT/NaN with a large value (e.g., 9999) 
        # to indicate a very long time/expired status, or use the median/mean.
        new_feature_name = 'Days_Since_' + self.date_col.replace('Date', '').strip().replace('_', '')
        
        # Days: If a date is missing, fill with the max observed days or a fixed large value.
        X_copy[new_feature_name] = time_delta.dt.days.fillna(9999) 
        
        # Drop the original date column
        return X_copy.drop(columns=[self.date_col])

# ----------------------------------------------------------------------
# 2. Assume Data Loading and Define Feature Lists
# ----------------------------------------------------------------------

# **ASSUMPTION:** The DataFrame 'df' containing all 50,000 records is already loaded.

# Separate features (X) and target (y)
X = df.drop('Need_Maintenance', axis=1)
y = df['Need_Maintenance']

# --- Define Ordinal Categories and their Order (from worst/least to best/most) ---

# Maintenance_History: Poor (0) -> Good (2)
M_HIST_ORDER = ['Poor', 'Average', 'Good']

# Owner_Type: Third (0) -> First (2)
OWNER_ORDER = ['Third', 'Second', 'First'] 

# Condition Features: Worn Out/Weak (0) -> New (2)
TIRE_ORDER = ['Worn Out', 'Good', 'New'] 
BRAKE_ORDER = ['Worn Out', 'Good', 'New'] 
BATTERY_ORDER = ['Weak', 'Good', 'New'] 

# List of columns and their corresponding order for the OrdinalEncoder
ordinal_features = [
    'Maintenance_History', 
    'Owner_Type', 
    'Tire_Condition', 
    'Brake_Condition', 
    'Battery_Status'
]
ordinal_categories = [
    M_HIST_ORDER,
    OWNER_ORDER,
    TIRE_ORDER,
    BRAKE_ORDER,
    BATTERY_ORDER
]

# --- Define remaining feature types ---
date_features = ['Last_Service_Date', 'Warranty_Expiry_Date']

numerical_features = [
    'Mileage', 'Reported_Issues', 'Vehicle_Age', 'Engine_Size', 
    'Odometer_Reading', 'Insurance_Premium', 'Service_History', 
    'Accident_History', 'Fuel_Efficiency'
]

nominal_features = [
    'Vehicle_Model', 
    'Fuel_Type', 
    'Transmission_Type'
]

# ----------------------------------------------------------------------
# 3. Create Preprocessor and Pipeline
# ----------------------------------------------------------------------

# --- Preprocessor with ColumnTransformer ---

preprocessor = ColumnTransformer(
    transformers=[
        # 1. Date Feature Engineering
        ('date_service', DateToDaysTransformer(date_col='Last_Service_Date'), ['Last_Service_Date']),
        ('date_warranty', DateToDaysTransformer(date_col='Warranty_Expiry_Date'), ['Warranty_Expiry_Date']),

        # 2. Ordinal Encoding (ROBUST: handles unknown categories by setting them to -1)
        ('ord', OrdinalEncoder(
            categories=ordinal_categories, 
            handle_unknown='use_encoded_value', 
            unknown_value=-1
        ), ordinal_features),
        
        # 3. One-Hot Encoding (ROBUST: ignores unknown categories)
        ('nom', OneHotEncoder(
            handle_unknown='ignore', 
            sparse_output=False
        ), nominal_features),

        # 4. Scale remaining numerical features (includes the new 'Days_Since' features after transformation)
        # Note: We must specify all numerical columns (including the new ones) to ensure scaling is applied correctly.
        # Since the new features are not known yet, we use a placeholder of numerical_features and rely on 
        # the ColumnTransformer to pass the data correctly after the date transformer steps. 
        # A simpler way is to use 'select_dtypes' after the date transformation, but that is difficult in a single pipeline.
        # Given the structure, we pass the original numerical columns here.
        ('scale', StandardScaler(), numerical_features)
    ],
    # Drop all other columns (ensures only transformed columns are passed)
    remainder='drop' 
)

# --- Full Machine Learning Pipeline ---

rf_full_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(
        n_estimators=200, 
        random_state=42,
        class_weight='balanced', 
        n_jobs=-1          # Use all processor cores
    ))
])

# ----------------------------------------------------------------------
# 4. Train and Evaluate the Model
# ----------------------------------------------------------------------

# Split the data into training and testing sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.3, 
    random_state=42,
    stratify=y # Stratify ensures the class balance is maintained in the split
)

print("Starting Random Forest Model Training on 50,000 records...")
# Train the final pipeline (handles all feature engineering and preprocessing)
rf_full_pipeline.fit(X_train, y_train) 
print("Model Training Complete.")

# Make Predictions
y_pred = rf_full_pipeline.predict(X_test)

# Evaluate the Model
print("\n--- Model Evaluation ---")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Optional: Feature Importance (requires slightly more complex code to extract names)
# To keep the code clean, we will only print the Raw Feature Importance array.
# print("\nRaw Feature Importances Array:", rf_full_pipeline['classifier'].feature_importances_)

Starting Random Forest Model Training on 50,000 records...
Model Training Complete.

--- Model Evaluation ---
Accuracy: 1.0

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      2851
           1       1.00      1.00      1.00     12149

    accuracy                           1.00     15000
   macro avg       1.00      1.00      1.00     15000
weighted avg       1.00      1.00      1.00     15000



In [15]:
# Assuming your DataFrame 'df' is loaded
numerical_features = df.select_dtypes(include=np.number).columns.tolist()
numerical_features.remove('Need_Maintenance') # Remove the target itself

print("Correlation with Need_Maintenance:")
print(df[numerical_features + ['Need_Maintenance']].corr()['Need_Maintenance'].sort_values(ascending=False))

Correlation with Need_Maintenance:
Need_Maintenance     1.000000
Reported_Issues      0.389273
Service_History      0.104155
Accident_History     0.080590
Odometer_Reading     0.007876
Insurance_Premium    0.004775
Fuel_Efficiency      0.001874
Vehicle_Age          0.000806
Mileage              0.000704
Engine_Size         -0.000326
Name: Need_Maintenance, dtype: float64


In [16]:
categorical_features = [
    'Maintenance_History', 'Tire_Condition', 'Brake_Condition', 
    'Battery_Status', 'Owner_Type', 'Vehicle_Model', 'Fuel_Type', 
    'Transmission_Type'
]

print("\n--- Cross-Tabulation (Categorical Feature Check) ---")
for col in categorical_features:
    # Print the distribution of the target variable for each category
    cross_tab = pd.crosstab(df[col], df['Need_Maintenance'], normalize='index')
    
    # We are looking for rows that show 1.0 in the '1' column (perfect prediction)
    # The sum of rows is 1.0
    if (cross_tab[1.0] == 1.0).any() or (cross_tab[0.0] == 1.0).any():
         print(f"\nPotential Leakage in: {col}")
         print(cross_tab)


--- Cross-Tabulation (Categorical Feature Check) ---

Potential Leakage in: Brake_Condition
Need_Maintenance         0         1
Brake_Condition                     
Good              0.287619  0.712381
New               0.282817  0.717183
Worn Out          0.000000  1.000000

Potential Leakage in: Battery_Status
Need_Maintenance         0         1
Battery_Status                      
Good              0.283304  0.716696
New               0.285979  0.714021
Weak              0.000000  1.000000
